In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
#The number of unique schools
num_uniq_schools = school_data_complete.school_name.nunique()
num_uniq_schools


15

In [3]:
#The number of students
num_students = len(school_data_complete)
num_students

39170

In [4]:
#The total budget
group_budget_df = school_data_complete.groupby('school_name')['budget'].mean().sum()
group_budget_df


24649428.0

In [5]:
#The average maths score
avg_maths_score = school_data_complete['maths_score'].mean()
avg_maths_score



70.33819249425581

In [6]:
#The average reading score
avg_read_score = school_data_complete['reading_score'].mean()
avg_read_score

69.98013786060761

In [7]:
# %passing maths (the percentage of students who passed maths)
passed_maths = len(school_data_complete[school_data_complete['maths_score'] >= 50])
pct_passed_maths = (passed_maths / num_students) * 100
pct_passed_maths

86.07863160582077

In [8]:
# %passing reading (the percentage of students who passed reading)
passed_reading = len(school_data_complete[school_data_complete['reading_score'] >= 50])
pct_passed_reading = (passed_reading / num_students) * 100
pct_passed_reading

84.42685728874139

In [9]:
# %overall passing (the percentage of students who passed maths and reading)
combined_passed = len(school_data_complete[(school_data_complete['reading_score'] >= 50) \
                          & (school_data_complete['maths_score'] >= 50)]) * 100/num_students
combined_passed                     

72.80827163645647

In [10]:
area_summary = pd.DataFrame({"Total Schools":[num_uniq_schools],
                               "Total Students":[num_students],
                               "Total Budget":[group_budget_df],
                               "Average Maths Score":[avg_maths_score],
                               "Average Reading Score":[avg_read_score],
                               "% Passing Maths":[pct_passed_maths],
                               "% Passed Reading":[pct_passed_reading],
                               "% Overall Passing":[combined_passed]
                              })

area_summary['Total Budget'] = area_summary['Total Budget'].map('${:,.2f}'.format)

area_summary

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passed Reading,% Overall Passing
0,15,39170,"$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


In [11]:
school_data_complete['Average Maths pct'] = np.nan
school_data_complete['Per Student Budget'] = np.nan
school_data_complete['% Passing Maths'] = np.nan
school_data_complete['% Passing Reading'] = np.nan
school_data_complete['readanymark'] = np.nan

school_data_complete['bothgreater50'] = np.nan
school_data_complete['% Overall Passing'] = np.nan



In [12]:
school_data_complete['mthsgreater50'] = school_data_complete.groupby(['school_name'])['maths_score'].apply(lambda x: (x >= 50).astype(int).cumsum())
school_data_complete['mthsanymark'] = school_data_complete.groupby(['school_name'])['maths_score'].apply(lambda x: (x >= 0).astype(int).cumsum())

school_data_complete['readgreater50'] = school_data_complete.groupby(['school_name'])['reading_score'].apply(lambda x: (x >= 50).astype(int).cumsum())
school_data_complete['readanymark'] = school_data_complete.groupby(['school_name'])['reading_score'].apply(lambda x: (x >= 0).astype(int).cumsum())

school_data_complete['cumsum_both'] = (school_data_complete['maths_score'] >= 50) & (school_data_complete['reading_score'] >= 50)
school_data_complete['cumsum_both'] = school_data_complete.groupby('school_name')['cumsum_both'].cumsum()
school_data_complete['cumsum_both'] = school_data_complete['cumsum_both'].astype(int)

school_data_complete.rename(columns = {'type':'School type', 'size':'Total Students', 'budget':'Total School Budget','maths_score':'Average Maths Score', 'reading_score':'Average Reading Score'}, inplace=True)



In [13]:
new_school_summ_col_df = school_data_complete[['school_name','School type','Total Students','Total School Budget','Average Maths Score','Average Reading Score','Per Student Budget','% Passing Maths','% Passing Reading','Average Maths pct','mthsgreater50','mthsanymark','readgreater50','readanymark','% Overall Passing','cumsum_both']]
per_school_summary = new_school_summ_col_df.groupby(['school_name','School type']).mean()                                                                          
                                  
                                      

In [14]:
#Working on School Summary
per_school_summary['Per Student Budget'] =  per_school_summary['Per Student Budget'].astype(float)
per_school_summary['Per Student Budget'] = per_school_summary['Total School Budget']/per_school_summary['Total Students']
per_school_summary['% Passing Maths'] = (per_school_summary['mthsgreater50']/per_school_summary['mthsanymark']) * 100 
per_school_summary['% Passing Reading'] = (per_school_summary['readgreater50']/per_school_summary['readanymark'] * 100)
#per_school_summary['% Overall Passing'] = (per_school_summary['cumsum_both']/per_school_summary['readanymark'] * 100)

per_school_summary['% Overall Passing'] = (round(per_school_summary['cumsum_both'].round(0)) / per_school_summary['readanymark'] * 100)


In [15]:
per_school_summary=per_school_summary.drop('Average Maths pct',axis=1)
per_school_summary=per_school_summary.drop('mthsgreater50',axis=1)
per_school_summary=per_school_summary.drop('mthsanymark',axis=1)
per_school_summary=per_school_summary.drop('readgreater50',axis=1)
per_school_summary=per_school_summary.drop('readanymark',axis=1)
spending_summary = per_school_summary.copy()
type_summary = per_school_summary.copy()


In [16]:
#Here is the School Summary for marking
per_school_summary['Total School Budget'] = per_school_summary['Total School Budget'].map('${:,.2f}'.format)
per_school_summary['Per Student Budget'] = per_school_summary['Per Student Budget'].map('${:,.2f}'.format)
per_school_summary['Total Students'] = per_school_summary['Total Students'].map('{:,.0f}'.format)
per_school_summary=per_school_summary[['Total Students','Total School Budget','Per Student Budget','Average Maths Score','Average Reading Score','% Passing Maths','% Passing Reading','% Overall Passing']] 
per_school_summary

,,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,School type,,,,,,,,
Bailey High School,Government,"4,976","$3,124,928.00",$628.00,72.352894,71.008842,91.814154,87.230085,80.208961
Cabrera High School,Independent,"1,858","$1,081,356.00",$582.00,71.657158,71.359526,90.500755,89.143092,80.258203
Figueroa High School,Government,"2,949","$1,884,411.00",$639.00,68.698542,69.077993,80.983706,82.896334,67.118644
Ford High School,Government,"2,739","$1,763,916.00",$644.00,69.091274,69.572472,82.146369,81.939357,66.861314
Griffin High School,Independent,"1,468","$917,500.00",$625.00,71.788147,71.245232,91.297626,88.083424,80.871341
Hernandez High School,Government,"4,635","$3,022,020.00",$652.00,68.874865,69.186408,80.542827,81.921401,66.177739
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.194336,87.471820,77.570093
Huang High School,Government,"2,917","$1,910,635.00",$655.00,68.935207,68.910525,81.435761,81.103963,66.209733
Johnson High School,Government,"4,761","$3,094,650.00",$650.00,68.843100,69.039277,82.295947,82.297041,67.828643


In [17]:
#Top performing schools by Overall Passing 
#I realise there is some % innacuracy and I cannot figure out why
top_schools = per_school_summary.sort_values(by = '% Overall Passing',ascending = False)
top_schools.head(5)

,,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,School type,,,,,,,,
Griffin High School,Independent,"1,468","$917,500.00",$625.00,71.788147,71.245232,91.297626,88.083424,80.871341
Cabrera High School,Independent,"1,858","$1,081,356.00",$582.00,71.657158,71.359526,90.500755,89.143092,80.258203
Bailey High School,Government,"4,976","$3,124,928.00",$628.00,72.352894,71.008842,91.814154,87.230085,80.208961
Rodriguez High School,Government,"3,999","$2,547,363.00",$637.00,72.047762,70.935984,91.038322,87.490285,79.650000
Wright High School,Independent,"1,800","$1,049,400.00",$583.00,72.047222,70.969444,91.662040,86.681967,79.511383


In [18]:
#Bottom performing schools by Overall Passing 
#I have caught cumsum adding small fractions to the count so I rounded it.
bottom_schools = per_school_summary.sort_values(by = '% Overall Passing',ascending = True)
bottom_schools.head(5)

,,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,School type,,,,,,,,
Hernandez High School,Government,"4,635","$3,022,020.00",$652.00,68.874865,69.186408,80.542827,81.921401,66.177739
Huang High School,Government,"2,917","$1,910,635.00",$655.00,68.935207,68.910525,81.435761,81.103963,66.209733
Ford High School,Government,"2,739","$1,763,916.00",$644.00,69.091274,69.572472,82.146369,81.939357,66.861314
Figueroa High School,Government,"2,949","$1,884,411.00",$639.00,68.698542,69.077993,80.983706,82.896334,67.118644
Johnson High School,Government,"4,761","$3,094,650.00",$650.00,68.843100,69.039277,82.295947,82.297041,67.828643


In [19]:
#school_data_complete.head()
maths_score_by_year = school_data_complete[['Average Maths Score','school_name','year']].groupby(['school_name','year']).mean().unstack()
maths_score_by_year

Average Maths Score                                 
year                                   9          10         11         12
school_name                                                               
Bailey High School              72.493827  71.897498  72.374900  72.675097
Cabrera High School             72.321970  72.437768  71.008299  70.604712
Figueroa High School            68.477804  68.331586  68.811001  69.325282
Ford High School                69.021609  69.387006  69.248862  68.617811
Griffin High School             72.789731  71.093596  71.692521  71.469178
Hernandez High School           68.586831  68.867156  69.154412  68.985075
Holden High School              70.543307  75.105263  71.640777  73.409639
Huang High School               69.081754  68.533246  69.431345  68.639316
Johnson High School             69.469286  67.990220  68.637730  69.287393
Pena High School                71.996364  72.396000  72.523438  71.187845
Rodriguez High School           71.940722  71.779808  72.364811  72.154626
Shelton High School             72.932075  72.506696  70.097087  72.331536
Thomas High School              69.234273  70.057007  69.657831  69.369822
Wilson High School              69.212361  69.455446  68.378965  69.787472
Wright High School              71.741176  72.179012  73.275862  70.848238

In [20]:
reading_scores_by_year = school_data_complete[['Average Reading Score','school_name','year']].groupby(['school_name','year']).mean().unstack()
reading_scores_by_year

Average Reading Score                                 
year                                     9          10         11         12
school_name                                                                 
Bailey High School                70.901920  70.848265  70.317346  72.195525
Cabrera High School               71.172348  71.328326  71.201245  71.856021
Figueroa High School              70.261682  67.677588  69.152327  69.082126
Ford High School                  69.615846  68.988701  70.735964  68.849722
Griffin High School               72.026895  70.746305  72.385042  69.434932
Hernandez High School             68.477569  70.621842  68.418199  69.244136
Holden High School                71.598425  71.096491  73.310680  70.481928
Huang High School                 68.670616  69.516297  68.740638  68.671795
Johnson High School               68.719286  69.295029  69.969115  67.992521
Pena High School                  70.949091  72.324000  71.703125  71.513812
Rodriguez High School             70.902921  70.137500  71.424453  71.414449
Shelton High School               70.715094  69.879464  71.150485  69.070081
Thomas High School                69.672451  69.741093  70.749398  68.730769
Wilson High School                68.683043  68.412541  68.796327  69.888143
Wright High School                71.823529  71.386831  70.475862  69.821138

In [21]:

spending_summary['Per Student Budget'] = pd.to_numeric(spending_summary['Per Student Budget'], errors='coerce')



In [22]:
#scores by school spending
bins = [0, 585, 630, 645, 680]
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]
spending_summary.loc[:, "Spending Ranges (Per Student)"] = pd.cut(spending_summary["Per Student Budget"], bins, labels=group_names, include_lowest=True)
spending_maths_scores = spending_summary.groupby("Spending Ranges (Per Student)")["Average Maths Score"].mean()
spending_reading_scores = spending_summary.groupby("Spending Ranges (Per Student)")["Average Reading Score"].mean()
spending_passing_maths = spending_summary.groupby("Spending Ranges (Per Student)")["% Passing Maths"].mean()
spending_passing_reading = spending_summary.groupby("Spending Ranges (Per Student)")["% Passing Reading"].mean()
overall_passing_spending = spending_summary.groupby("Spending Ranges (Per Student)")["% Overall Passing"].mean()
spending_summary = spending_summary.groupby(['Spending Ranges (Per Student)']).mean()[["Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]]
spending_summary["Spending Range"] = group_names
spending_summary.set_index("Spending Range", inplace=True)

spending_summary['Average Maths Score'] = spending_summary['Average Maths Score'].map('{:,.2f}'.format)
spending_summary['Average Reading Score'] = spending_summary['Average Reading Score'].map('{:,.2f}'.format)
spending_summary['% Passing Maths'] = spending_summary['% Passing Maths'].map('{:,.2f}'.format)
spending_summary['% Passing Reading'] = spending_summary['% Passing Reading'].map('{:,.2f}'.format)
spending_summary['% Overall Passing'] = spending_summary['% Overall Passing'].map('{:,.2f}'.format)

spending_summary




,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Range,,,,,
<$585,71.36,70.72,88.56,86.29,76.34
$585-630,72.07,71.03,91.56,86.96,79.60
$630-645,69.85,69.84,84.57,83.76,70.86
$645-680,68.88,69.05,81.42,81.77,66.74


In [23]:
#scores by school size
per_school_summary["Total Students"] = per_school_summary["Total Students"].str.replace(",", "").astype(int)

bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
per_school_summary.loc[:, "School Size"] = pd.cut(per_school_summary["Total Students"], bins, labels=group_names, include_lowest=True)
spending_maths_scores = per_school_summary.groupby("School Size")["Average Maths Score"].mean()
spending_reading_scores = per_school_summary.groupby("School Size")["Average Reading Score"].mean()
spending_passing_maths = per_school_summary.groupby("School Size")["% Passing Maths"].mean()
spending_passing_reading = per_school_summary.groupby("School Size")["% Passing Reading"].mean()
overall_passing_spending = per_school_summary.groupby("School Size")["% Overall Passing"].mean()
size_summary = per_school_summary.groupby(['School Size']).mean()[["Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]]
size_summary["School Size"] = group_names
size_summary.set_index("School Size", inplace=True)
size_summary

#spending_stats

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.401383,86.438030,77.829699
Medium (1000-2000),71.421650,70.720164,89.814550,86.746834,77.934696
Large (2000-5000),69.751809,69.576052,84.143571,83.340681,70.261695


In [24]:
#I have copied school_summary to type_summary earlier above 
#as there were columns I wanted
type_summary=type_summary.drop('Per Student Budget',axis=1)
type_summary=type_summary.drop('cumsum_both',axis=1)
type_summary=type_summary.drop('Total School Budget',axis=1)
type_summary=type_summary.drop('Total Students',axis=1)

grouped_data = type_summary.groupby('School type').mean()
grouped_data
#final graph Scores by School Type

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School type,,,,,
Government,69.834806,69.675929,84.322441,83.554066,70.579291
Independent,71.368822,70.718933,89.095874,86.057151,76.671426
